# LangChain 核心模块 Agent - OpenAI Function


In [1]:
%%capture --no-stderr
# 去掉pip install 使用项目依赖版本
# %pip install -U langchain

In [2]:
from langchain_openai import ChatOpenAI

# 使用 gpt-4o-mini
chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [3]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

In [4]:
from langchain.schema import SystemMessage
from langchain.agents import OpenAIFunctionsAgent

system_message = SystemMessage(content="你是非常强大的AI助手，但在计算单词长度方面不擅长。")
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)

In [5]:
agent = OpenAIFunctionsAgent(llm=chat_model, tools=tools, prompt=prompt)

/tmp/ipykernel_1745891/1839730416.py:1: LangChainDeprecationWarning: The class `OpenAIFunctionsAgent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use create_openai_functions_agent instead.
  agent = OpenAIFunctionsAgent(llm=chat_model, tools=tools, prompt=prompt)


In [6]:
from langchain.agents import AgentExecutor

# 实例化 OpenAIFunctionsAgent
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [7]:
agent_executor.invoke("单词“educa”中有多少个字母?")



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5单词“educa”中有5个字母。

> Finished chain.


{'input': '单词“educa”中有多少个字母?', 'output': '单词“educa”中有5个字母。'}

In [8]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=MEMORY_KEY)]
)

In [9]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)

In [11]:
agent = OpenAIFunctionsAgent(llm=chat_model, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

In [12]:
agent_executor.invoke("单词“educa”中有多少个字母?")



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5单词“educa”中有5个字母。

> Finished chain.


{'input': '单词“educa”中有多少个字母?',
 'chat_history': [HumanMessage(content='单词“educa”中有多少个字母?'),
  AIMessage(content='单词“educa”中有5个字母。')],
 'output': '单词“educa”中有5个字母。'}

In [13]:
agent_executor.invoke("那是一个真实的单词吗？")



> Entering new AgentExecutor chain...
“educa”并不是一个常见的英语单词。它可能是“educate”的一部分或某个特定名称的缩写，但在标准英语词汇中并不存在。如果你有其他单词需要确认或查询，请告诉我！

> Finished chain.


{'input': '那是一个真实的单词吗？',
 'chat_history': [HumanMessage(content='单词“educa”中有多少个字母?'),
  AIMessage(content='单词“educa”中有5个字母。'),
  HumanMessage(content='那是一个真实的单词吗？'),
  AIMessage(content='“educa”并不是一个常见的英语单词。它可能是“educate”的一部分或某个特定名称的缩写，但在标准英语词汇中并不存在。如果你有其他单词需要确认或查询，请告诉我！')],
 'output': '“educa”并不是一个常见的英语单词。它可能是“educate”的一部分或某个特定名称的缩写，但在标准英语词汇中并不存在。如果你有其他单词需要确认或查询，请告诉我！'}